In [205]:
 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as mpatches
from mpl_toolkits.basemap import Basemap
import zipcode
%matplotlib inline 

In [206]:
master = pd.DataFrame.from_csv("nps_all.csv")
master.head(7)

,Zip,Best Season,2nd Best Season,3rd Best Season,4th Best Season,Off Season
Abraham Lincoln Birthplace NHP,42748,NaN,NaN,NaN,NaN,NaN
Acadia NP,4660,NaN,NaN,NaN,NaN,NaN
Adams NHP,2169,NaN,NaN,NaN,NaN,NaN
African Burial Ground NM,10007,NaN,NaN,NaN,NaN,NaN
Agate Fossil Beds NM,69346,NaN,NaN,NaN,NaN,NaN
Alibates Flint Quarries NM,79036,NaN,NaN,NaN,NaN,NaN
Allegheny Portage Railroad NHS,16641,NaN,NaN,NaN,NaN,NaN


In [207]:
def num_month(month):
    months = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
    return months.index(month) + 1

In [208]:
files = master.index


for i in range(0, len(files)):
    part = pd.DataFrame.from_csv("tables/" + files[i] + ".csv")
    part = part.drop('Textbox5', 1)
    avgs = pd.DataFrame(part.ix[0].astype(float))
    avgs = avgs.sort(columns='Average')
    ranking = avgs.index
    best = num_month(ranking[0])
    best2 = num_month(ranking[1])
    best3 = num_month(ranking[2])
    best4 = num_month(ranking[3])
    worst = num_month(ranking[-1])
    master.ix[i,1] = best
    master.ix[i,2] = best2
    master.ix[i,3] = best3
    master.ix[i,4] = best4
    master.ix[i,5] = worst
    
master
    

C:\Users\bborlas\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:8: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,Zip,Best Season,2nd Best Season,3rd Best Season,4th Best Season,Off Season
Abraham Lincoln Birthplace NHP,42748,1,12,2,11,7
Acadia NP,4660,12,1,2,3,8
Adams NHP,2169,1,12,2,3,7
African Burial Ground NM,10007,1,12,2,11,7
Agate Fossil Beds NM,69346,2,1,12,11,7
Alibates Flint Quarries NM,79036,1,12,2,11,5
Allegheny Portage Railroad NHS,16641,2,1,12,11,7
Amistad NRA,78840,12,11,1,10,3
Andersonville NHS,31711,1,12,8,11,5
Andrew Johnson NHS,37743,2,1,11,9,5


In [209]:
def color_translate(color_num):
    colors = ['red','coral','orange','gold','yellow','greenyellow','green','teal','blue','slateblue','blueviolet','darkmagenta']
    return colors[color_num - 1]

color_translate(int(master.ix[0,1]))

'red'

In [210]:
master['Best color'] = master['Best Season'].astype(int)
master['Best color'] = master['Best color'].map(color_translate)
master.head()

,Zip,Best Season,2nd Best Season,3rd Best Season,4th Best Season,Off Season,Best color
Abraham Lincoln Birthplace NHP,42748,1,12,2,11,7,red
Acadia NP,4660,12,1,2,3,8,darkmagenta
Adams NHP,2169,1,12,2,3,7,red
African Burial Ground NM,10007,1,12,2,11,7,red
Agate Fossil Beds NM,69346,2,1,12,11,7,coral


In [211]:
#zipcode.isequal('04660').lat

master['Zip'] = master['Zip'].astype(str)
master['Zip'] = master['Zip'].map(lambda x: x.rjust(5, "0"))

master

,Zip,Best Season,2nd Best Season,3rd Best Season,4th Best Season,Off Season,Best color
Abraham Lincoln Birthplace NHP,42748,1,12,2,11,7,red
Acadia NP,04660,12,1,2,3,8,darkmagenta
Adams NHP,02169,1,12,2,3,7,red
African Burial Ground NM,10007,1,12,2,11,7,red
Agate Fossil Beds NM,69346,2,1,12,11,7,coral
Alibates Flint Quarries NM,79036,1,12,2,11,5,red
Allegheny Portage Railroad NHS,16641,2,1,12,11,7,coral
Amistad NRA,78840,12,11,1,10,3,darkmagenta
Andersonville NHS,31711,1,12,8,11,5,red
Andrew Johnson NHS,37743,2,1,11,9,5,coral


In [212]:
def make_lat(zobject):
    if zobject is not None:
        return zobject.lat
    else:
        return 0.0

def make_lon(zobject):
    if zobject is not None:
        return zobject.lon
    else:
        return 0.0
    
master['ZObjects'] = master['Zip'].map(lambda x: zipcode.isequal(x))
master['Lat'] = master['ZObjects'].map(lambda x: make_lat(x))
master['Lon'] = master['ZObjects'].map(lambda x: make_lon(x))
master

,Zip,Best Season,2nd Best Season,3rd Best Season,4th Best Season,Off Season,Best color,ZObjects,Lat,Lon
Abraham Lincoln Birthplace NHP,42748,1,12,2,11,7,red,<Zip: 42748>,37.56,-85.73
Acadia NP,04660,12,1,2,3,8,darkmagenta,<Zip: 04660>,44.31,-68.36
Adams NHP,02169,1,12,2,3,7,red,<Zip: 02169>,42.26,-71.00
African Burial Ground NM,10007,1,12,2,11,7,red,<Zip: 10007>,40.71,-73.99
Agate Fossil Beds NM,69346,2,1,12,11,7,coral,<Zip: 69346>,42.68,-103.88
Alibates Flint Quarries NM,79036,1,12,2,11,5,red,<Zip: 79036>,35.64,-101.59
Allegheny Portage Railroad NHS,16641,2,1,12,11,7,coral,<Zip: 16641>,40.48,-78.55
Amistad NRA,78840,12,11,1,10,3,darkmagenta,<Zip: 78840>,29.37,-100.89
Andersonville NHS,31711,1,12,8,11,5,red,<Zip: 31711>,32.19,-84.14
Andrew Johnson NHS,37743,2,1,11,9,5,coral,<Zip: 37743>,36.16,-82.81


In [213]:
master[(master.Lat == 0.0) & (master.Lon == 0.0)]

,Zip,Best Season,2nd Best Season,3rd Best Season,4th Best Season,Off Season,Best color,ZObjects,Lat,Lon
Antietam NB,21782,1,2,3,11,7,red,None,0,0
Arlington House The RE Lee MEM,22211,1,2,12,11,5,red,None,0,0
Assateague Island NS,21811,12,1,2,3,7,darkmagenta,None,0,0
Cape Hatteras NS,27915,2,1,12,11,7,coral,None,0,0
Cape Lookout NS,28531,2,1,3,12,7,coral,None,0,0
Carl Sandburg Home NHS,28731,2,1,12,9,10,coral,None,0,0
Catoctin Mountain Park,21788,1,2,12,11,8,red,None,0,0
Charles Pinckney NHS,29464,12,1,11,2,4,darkmagenta,None,0,0
Chattahoochee River NRA,30350,1,10,12,11,6,red,None,0,0
Chesapeake & Ohio Canal NHP,20854,2,1,12,3,8,coral,None,0,0


In [214]:
master.to_csv("Master.csv")